In [1]:
pip install tensorflow pyyaml opencv-python pillow numpy pandas 

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.17.0-cp310-cp310-win_amd64.whl (2.0 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl (161 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
  Using cached pillow-10.4.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached tensorflow_intel-2.17.0-cp310-cp310-win_amd64.whl (385.0 MB)
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
  Using cached grpcio-1.66.1-cp310-cp310-win_amd64.whl (4.3 MB)
  Using cached protobuf-4.25.4-cp310-abi3-win_amd64.whl (413 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl (5.5 MB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl (1.5 MB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Usi

You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install tensorflow-addons

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow_addons-0.22.0-cp310-cp310-win_amd64.whl (719 kB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [10]:
import tensorflow as tf
import tensorflow_addons as tfa

# Определение класса pFBeta
class pFBeta(tf.keras.metrics.Metric):
    """Compute overall probabilistic F-beta score."""
    def __init__(self, beta=1, epsilon=1e-5, name='pF1', **kwargs):
        super().__init__(name=name, **kwargs)
        self.beta = beta
        self.epsilon = epsilon
        self.pos = self.add_weight(name='pos', initializer='zeros')
        self.ctp = self.add_weight(name='ctp', initializer='zeros')
        self.cfp = self.add_weight(name='cfp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 0, 1)
        pos = tf.reduce_sum(y_true)
        ctp = tf.reduce_sum(y_pred[y_true == 1])
        cfp = tf.reduce_sum(y_pred[y_true == 0])
        self.pos.assign_add(pos)
        self.ctp.assign_add(ctp)
        self.cfp.assign_add(cfp)

    def result(self):
        beta_squared = self.beta * self.beta
        c_precision = self.ctp / (self.ctp + self.cfp + self.epsilon)
        c_recall = self.ctp / (self.pos + self.epsilon)
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return tf.cond(c_precision > 0 and c_recall > 0, lambda: result, lambda: 0.0)

# Определение функции pfbeta_thr
def pfbeta_thr(labels, preds):
    thrs = tf.range(0, 1, 0.05)
    best_score = tf.constant(0, dtype=tf.float32)
    for thr in thrs:
        score = pfbeta_tf(labels, tf.cast(preds > thr, tf.float32))
        best_score = tf.cond(score > best_score, lambda: score, lambda: best_score)
    return best_score

pfbeta_thr.__name__ = 'pF1_thr'

# Определение custom_objects с использованием custom_object_scope
custom_objects = {
    "Addons>SigmoidFocalCrossEntropy": tfa.losses.SigmoidFocalCrossEntropy(),
    "pFBeta": pFBeta,
    "pF1_thr": pfbeta_thr
}

# Загрузка обученной модели с указанием custom_objects через custom_object_scope
model_path = r'C:\Users\marus\Desktop\Breast Cancer Ai\fold-2.h5'  # путь к вашей модели
with tf.keras.utils.custom_object_scope(custom_objects):
    model = tf.keras.models.load_model(model_path)



ModuleNotFoundError: No module named 'keras.src.engine'

In [11]:
# Загрузка изображения с правильным размером

#CHANGE 'IMG_PATH' TO URL OR SMTH ELSE
img_path = r'images\cancer.jpg'

img = tf.keras.preprocessing.image.load_img(img_path, target_size=(1024, 512))
img_array = tf.keras.preprocessing.image.img_to_array(img)

# Нормализация изображения
img_array = img_array / 255.0  # Если модель обучалась на нормализованных данных

img_array = tf.expand_dims(img_array, axis=0)  # Добавляем размерность для батча

# Предсказание
predictions = model.predict(img_array)
print("Raw prediction:", predictions[0])

# Обработка результатов
threshold = 0.4  # Порог вероятности для классификации
result = "Cancer detected" if predictions[0] > threshold else "No cancer detected"
print(result)

NameError: name 'model' is not defined